An interactive DFT visualisation, inspired by: https://youtu.be/spUNpyF58BY?si=ywZXRa4QD9QePe56

In [1]:
from ipywidgets import interact, FloatSlider, IntSlider, Layout, Checkbox
# import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
# import itertools
from scipy.fft import fft, fftfreq
from matplotlib.gridspec import GridSpec
import os
from PIL import Image
import glob

In [2]:
# Some parameters
max_frequency = 10
# TODO: put more parameters here

In [3]:
def get_sines(samplerate, frequencies = [1], seconds=3):
  time = np.linspace(0, seconds, int(samplerate*seconds))
  sines = np.array([np.sin(2*np.pi*f*time) for f in frequencies])
  return time, sines

def get_complex_coordinates(time, sine, f_wrap):
  return np.array([-s*np.exp(complex(0,1)*t*2*np.pi*f_wrap) for t, s in zip(time, sine)])

def plot_signal_creator(time, component_sines, cummulative_sine, discrete=False):
  plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.Pastel1.colors)  # Changing colours to pastel

  # Plot each individual sine
  if len(component_sines) > 1:  # No need to plot just one component, as it is identical to the cummulative plot
    for i, sine in enumerate(component_sines):
      plt.plot(time, sine, '--', label= f"f_{i}")

  # Plot the cummulative of the individual sines
  if discrete:
    plt.plot(time, cummulative_sine, '.', c='purple', label="Signal")
  else:
    plt.plot(time, cummulative_sine, c='purple', label="Signal")
  plt.xlabel("Times (s)")
  plt.ylabel("Amplitude")
  plt.title("Created signal (sum of sines(s)) in time domain (x[n])")
  # plt.legend()
  plt.grid(True)

def plot_complex_plane(time, sine, f_wrap, discrete=False):
  complex_coordinates = get_complex_coordinates(time, sine, f_wrap)
  centre_of_mass = complex_coordinates.mean()

  # Thicker horizontal and vertical 0 lines
  plt.axhline(0, color='black', linewidth=1)  # Horizontal line
  plt.axvline(0, color='black', linewidth=1)  # Vertical line

  if discrete:
    plt.gca().plot(complex_coordinates.real, complex_coordinates.imag, '.', c='purple')
  else:
    plt.gca().plot(complex_coordinates.real, complex_coordinates.imag, c='purple')
  plt.gca().scatter(centre_of_mass.real, centre_of_mass.imag, c='red', zorder=3)
  plt.ylim(-3, 3)
  plt.xlim(-3, 3)
  plt.xlabel("Real")
  plt.ylabel("Imaginary")
  plt.title("Same as above, but in polar coordinates\n i.e. \"wrapped around complex plane\"")  
  plt.grid(True)

# Here I implemented the actual Dicrete Fourier Transform, it is good for educational purposes and for comparison with FFT. It is usuable, but slower than the FFT.
# Below it is commented out, as it is not used in the visualisation by default. If you want to use it, uncomment the function below.
def plot_Ewouts_DFT(time, sine, f_wrap, samplerate, discrete=False):
  # Think of f_wraps as the frequency axis in the spectrum plot.
  f_wraps = np.linspace(0, 0.5*samplerate, int(0.5*samplerate*np.max(time)), endpoint=False)
  complex_sums = np.array([get_complex_coordinates(time, sine, f_wrap).sum() for f_wrap in f_wraps])
  amplitudes = np.abs(complex_sums)
  if discrete:
    plt.gca().plot(f_wraps, amplitudes, '.', c='purple')
  else:
    plt.gca().plot(f_wraps, amplitudes, c='purple')
  plt.title("Created Signal in Frequency Domain (frequency vs np.abs(X[k]))")
  plt.gca().plot([f_wrap, f_wrap], [0, np.max(amplitudes)], c='red')
  plt.gca().get_yaxis().set_visible(False)
  plt.ylabel('Amplitude')
  plt.xlabel('Frequency (Hz)')

def plot_FFT(sine, f_wrap, samplerate, discrete=False):
  # Calculating parameters FFT
  period = 1/samplerate
  n_samples = len(sine)

  # FFT
  yf = fft(sine)
  xf = fftfreq(n_samples, period)[:n_samples//2]
  # xf = fftfreq(n_samples, period)[:n_samples]

  # Plot Frequency Domain
  if discrete:
    plt.gca().plot(xf, 2.0/n_samples * np.abs(yf[0:n_samples//2]), '.', c='purple')
  else:
    plt.gca().plot(xf, 2.0/n_samples * np.abs(yf[0:n_samples//2]), c='purple')

  # plt.gca().plot(xf, 2.0/n_samples * np.abs(yf), c='purple')
  plt.gca().plot([f_wrap, f_wrap], [0, 1], c='red')
  plt.title("Created Signal in Frequency Domain (frequency vs np.abs(X[k])), using scipy.fft")
  plt.xlabel("Frequency (Hz)")
  plt.ylabel("Amplitude")

def plot_fourier_visualisation(discrete, seconds, samplerate, f_1, f_2, f_3, f_wrap, plt_show=True):
  frequencies = [f_1, f_2, f_3]  # Can not pass the frequencies as a list, due to widget unpacking in interact()
  frequencies = [f for f in frequencies if f != 0 and f is not None]  # Removing all frequencies that equal 0
  time, component_sines = get_sines(samplerate=samplerate, frequencies=frequencies, seconds=seconds)  # Create all the desired component signals
  cummulative_sine = component_sines.sum(axis=0)
  cummulative_sine = cummulative_sine

  # plt.figure(figsize=(32, 10))  # Resizing, for aesthetic purposes

  # plt.subplots(2, 2, figsize=(16,3), gridspec_kw={'width_ratios': [1, 3]})
  plt.figure(figsize=(16, 6))  # Set the figure size for all subplots

  gs = GridSpec(2, 2, height_ratios=[1, 1], width_ratios=[1, 4])

  plt.subplot(gs[0, :])  # First subplot spanning both columns
  plot_signal_creator(time, component_sines, cummulative_sine, discrete)
  plt.subplot(gs[1, 0])  # Second subplot
  plot_complex_plane(time, cummulative_sine, f_wrap, discrete)
  plt.subplot(gs[1, 1])  # Third subplot
  # plot_Ewouts_DFT(time, cummulative_sine, f_wrap, samplerate, discrete)
  plot_FFT(cummulative_sine, f_wrap, samplerate, discrete)
  if plt_show:
    plt.tight_layout()  # Adjust the layout to prevent overlap
    plt.show()

In [4]:
# Changing the layout sizes of the sliders (cosmetic/legibility changes)
widget_layout = Layout(width='500px')
widget_style = {'description_width': '150px'}

# The interactive sliders for all the parameters
discrete_widget = Checkbox(value=False, description='Discrete', layout=widget_layout, style=widget_style)
duration_widget = FloatSlider(min=0.5, max=5, step=0.5, value=1, description="Duration (s)", layout=widget_layout, style=widget_style)
samplerate_widget = IntSlider(min=5, max=500, step=1, value=100, description="Samplerate (samples/s)", layout=widget_layout, style=widget_style)
f_1_widget = FloatSlider(min=0, max=max_frequency, step=1, value=1, description="Sine 1 Frequency (Hz)", layout=widget_layout, style=widget_style)
f_2_widget = FloatSlider(min=0, max=max_frequency, step=1, value=0, description="Sine 2 Frequency (Hz)", layout=widget_layout, style=widget_style)
f_3_widget = FloatSlider(min=0, max=max_frequency, step=1, value=0, description="Sine 3 Frequency (Hz)", layout=widget_layout, style=widget_style)
f_wrap_widget = FloatSlider(min=0, max=max_frequency, step=0.1, value=0, description="f_wrap (cycles/s)", layout=widget_layout, style=widget_style)

# Render the interactive plot created above
interact(plot_fourier_visualisation, discrete=discrete_widget, seconds=duration_widget, samplerate=samplerate_widget, f_1=f_1_widget, f_2=f_2_widget, f_3=f_3_widget, f_wrap=f_wrap_widget, continuous_update=False);

interactive(children=(Checkbox(value=False, description='Discrete', layout=Layout(width='500px'), style=Checkb…

https://prajwalsouza.github.io/Experiments/Fourier-Transform-Visualization.html


In [5]:
# # Create the "images" directory if it doesn't exist
# if not os.path.exists("images"):
#     os.makedirs("images")

# f_wraps = np.arange(0, 2, 0.01)

# for i, f_wrap in enumerate(f_wraps):
#     plt.figure()  # Create a new figure
#     plot_fourier_visualisation(discrete=False, seconds=1, samplerate=100, f_1=1, f_2=None, f_3=None, f_wrap=f_wrap, plt_show=False)
#     plt.tight_layout()  # Adjust the layout to prevent overlap

#     plt.savefig(f"images/plot_{i}.png")
#     plt.close()

# # List of pngs in the images directory, sorted by filename 
# images = sorted(glob.glob("images/*.png"), key=lambda x: int(os.path.splitext(os.path.basename(x))[0].split('_')[1])) 

# # Open images and save as GIF
# frames = [Image.open(image) for image in images]
# frames[0].save('fourier_visualisation.gif', format='GIF', append_images=frames[1:], save_all=True, duration=200, loop=0)

C:\Users\beeo\AppData\Local\Temp\ipykernel_23316\689131757.py:97: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(16, 6))  # Set the figure size for all subplots
C:\Users\beeo\AppData\Local\Temp\ipykernel_23316\45994061.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()  # Create a new figure


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [8]:
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
# from PIL import Image
import cv2

# Create the "images" directory if it doesn't exist
if not os.path.exists("images"):
    os.makedirs("images")

f_wraps = np.arange(0, 2, 0.01)

for i, f_wrap in enumerate(f_wraps):
    plt.figure()  # Create a new figure
    plot_fourier_visualisation(discrete=False, seconds=1.5, samplerate=100, f_1=1, f_2=None, f_3=None, f_wrap=f_wrap, plt_show=False)
    plt.tight_layout()  # Adjust the layout to prevent overlap

    plt.savefig(f"images/plot_{i}.png")
    plt.close()

# List of pngs in the images directory, sorted by filename 
images = sorted(glob.glob("images/*.png"), key=lambda x: int(os.path.splitext(os.path.basename(x))[0].split('_')[1])) 

# Define the codec and create VideoWriter object
frame = cv2.imread(images[0])
height, width, layers = frame.shape
video = cv2.VideoWriter('fourier_visualisation.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 10, (width, height))

for image in images:
    video.write(cv2.imread(image))

cv2.destroyAllWindows()
video.release()

C:\Users\beeo\AppData\Local\Temp\ipykernel_23316\689131757.py:97: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(16, 6))  # Set the figure size for all subplots
C:\Users\beeo\AppData\Local\Temp\ipykernel_23316\194564280.py:15: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()  # Create a new figure


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

# Discrete Fourier Transform

# $$ X[k] = \sum_{n=0}^{N-1} x[n]e^{- 2 \pi j \frac{kn}{N}} $$

Where $N$ is the quantity of samples and:

$k \in [0, ..., N-1]$


# Fourier Transform

$\displaystyle X(k) = \int_{-\infty}^{\infty} x(t)e^{- 2 \pi j k t} \,dt$
